<a href="https://colab.research.google.com/github/vandant1/MyPortfoliodotcom/blob/main/Fault_Detection_in_Transmission_Line4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Upload Files to Colab Temporary Storage
from google.colab import files
uploaded = files.upload()

Saving classData.csv to classData.csv
Saving detect_dataset.csv to detect_dataset.csv


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Step 2: Load and Inspect Datasets
# Load datasets
detect_dataset = pd.read_csv('detect_dataset.csv')
class_data = pd.read_csv('classData.csv')


In [ ]:
# Combine Datasets
detect_dataset['G'] = 0
detect_dataset['C'] = 0
detect_dataset['B'] = 0
detect_dataset['A'] = detect_dataset['Output (S)']
combined_data = pd.concat([detect_dataset, class_data], ignore_index=True)

In [ ]:
# Drop unnecessary columns
combined_data.drop(columns=['Unnamed: 7', 'Unnamed: 8'], inplace=True)

In [ ]:

# Fill missing values
combined_data.fillna(0, inplace=True)

In [ ]:
# Step 3: Add Fault Distance Column
Z_line = 0.021  # Impedance per unit length (Ohms per meter)

def calculate_fault_distance(row):
    return abs(row['Va']) / (abs(row['Ia']) * Z_line) if row['Ia'] != 0 else 0

combined_data['fault_distance'] = combined_data.apply(calculate_fault_distance, axis=1)
combined_data['fault_distance'] = combined_data['fault_distance'].clip(lower=0, upper=7)



In [ ]:
# Step 4: Normalize Features
features = ['Ia', 'Ib', 'Ic', 'Va', 'Vb', 'Vc', 'fault_distance']
scaler = StandardScaler()
combined_data[features] = scaler.fit_transform(combined_data[features])

In [ ]:
# Step 5: Prepare Data for Training
# Separate features and labels
X = combined_data[features]
y = combined_data[['G', 'C', 'B', 'A']]

In [ ]:
# One-Hot Encode Labels
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y)

In [ ]:
# Train-Test Split
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [ ]:
# Step 6: Calculate Class Weights
y_train_original = np.argmax(y_train_encoded, axis=1)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_original), y=y_train_original)


In [ ]:
# Convert One-Hot Encoded Labels Back to Class Indices
y_train_single = np.argmax(y_train_encoded, axis=1)  # Convert to single column
y_test_single = np.argmax(y_test_encoded, axis=1)

# Reinitialize CatBoost Model for Multi-Class Classification
cat_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    depth=8,
    loss_function='MultiClass',  # Multi-class classification
    eval_metric='Accuracy',
    class_weights=class_weights.tolist(),
    random_state=42,
    verbose=100
)

print("Starting Model Training...")
cat_model.fit(X_train, y_train_single, eval_set=(X_test, y_test_single), plot=True)

# Evaluate the Model
y_pred = cat_model.predict(X_test)
accuracy = accuracy_score(y_test_single, y_pred)
print(f"CatBoost Model Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:")
print(classification_report(y_test_single, y_pred))


Starting Model Training...


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.8201985	test: 0.8163906	best: 0.8163906 (0)	total: 65.8ms	remaining: 1m 5s
100:	learn: 0.8309464	test: 0.8269722	best: 0.8269722 (96)	total: 1.63s	remaining: 14.5s
200:	learn: 0.8342662	test: 0.8310982	best: 0.8310982 (197)	total: 3.85s	remaining: 15.3s
300:	learn: 0.8361485	test: 0.8275557	best: 0.8310982 (197)	total: 6.08s	remaining: 14.1s
400:	learn: 0.8375639	test: 0.8222628	best: 0.8310982 (197)	total: 7.6s	remaining: 11.4s
500:	learn: 0.8384313	test: 0.8188924	best: 0.8310982 (197)	total: 9.18s	remaining: 9.14s
600:	learn: 0.8394557	test: 0.8161054	best: 0.8310982 (197)	total: 10.7s	remaining: 7.14s
700:	learn: 0.8401492	test: 0.8114170	best: 0.8310982 (197)	total: 12.3s	remaining: 5.26s
800:	learn: 0.8407458	test: 0.8087811	best: 0.8310982 (197)	total: 13.9s	remaining: 3.44s
900:	learn: 0.8416617	test: 0.8035093	best: 0.8310982 (197)	total: 15.7s	remaining: 1.72s
999:	learn: 0.8424152	test: 0.8008734	best: 0.8310982 (197)	total: 20.1s	remaining: 0us

bestTest = 0.831

In [ ]:
# Evaluate the Model
y_pred = cat_model.predict(X_test)
accuracy = accuracy_score(y_test_single, y_pred)
print(f"CatBoost Model Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:")
print(classification_report(y_test_single, y_pred))


CatBoost Model Accuracy: 72.74%

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.67      0.80      3288
           1       0.39      0.99      0.56       685

    accuracy                           0.73      3973
   macro avg       0.69      0.83      0.68      3973
weighted avg       0.89      0.73      0.76      3973



In [ ]:
# Step 9: Save the Model
cat_model.save_model('catboost_fault_detection_model.cbm')
print("Model saved successfully!")


Model saved successfully!


In [ ]:
# Step 10: Feature Importance
print("Feature Importance:")
feature_importances = cat_model.get_feature_importance()
for feature, importance in zip(X.columns, feature_importances):
    print(f"{feature}: {importance:.2f}")


Feature Importance:
Ia: 15.37
Ib: 8.07
Ic: 14.78
Va: 9.63
Vb: 6.40
Vc: 29.61
fault_distance: 16.13


In [ ]:
# Step 11: Download Model (Optional)
files.download('catboost_fault_detection_model.cbm')

FileNotFoundError: Cannot find file: catboost_fault_detection_model.cbm